# Определение токсичных комментариев

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Введение" data-toc-modified-id="Введение-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Введение</a></span></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка" data-toc-modified-id="Загрузка-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Загрузка</a></span></li><li><span><a href="#Предобработка" data-toc-modified-id="Предобработка-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Предобработка</a></span><ul class="toc-item"><li><span><a href="#Очистка,-приведение-к-нижнему-регистру" data-toc-modified-id="Очистка,-приведение-к-нижнему-регистру-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Очистка, приведение к нижнему регистру</a></span></li><li><span><a href="#Лемматизация-текста" data-toc-modified-id="Лемматизация-текста-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Лемматизация текста</a></span></li><li><span><a href="#Разбивка-на-обучающую,-тестовую-выборки,-скоринг-TF-IDF" data-toc-modified-id="Разбивка-на-обучающую,-тестовую-выборки,-скоринг-TF-IDF-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Разбивка на обучающую, тестовую выборки, скоринг TF-IDF</a></span></li></ul></li></ul></li><li><span><a href="#Обучение-и-предсказания" data-toc-modified-id="Обучение-и-предсказания-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение и предсказания</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>LGBMClassifier</a></span></li></ul></li><li><span><a href="#Итоги-исследования" data-toc-modified-id="Итоги-исследования-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Итоги исследования</a></span></li></ul></div>

## Введение

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

**Цель** - построить модель определения токсичных комментариев со значением метрики качества *F1* не меньше 0.75. 

**Задачи:**

1. Загрузите и подготовьте данные.

В рамках подготовки выполним следующее:
* Выполняют токенизацию каждого текста, то есть его разбивают на слова;
* Слова лемматизируют: приводят к начальной словарной форме (более сложные модели, например, BERT, этого не требуют: они сами понимают формы слов);
* Текст очищают от стоп-слов и ненужных символов;
* Затем предложения передают модели, которая переводит их в векторные представления. Для этого модель обращается к составленному заранее словарю токенов. На выходе для каждого текста образуются векторы заданной длины.

2. Обучите разные модели. 
3. Подвести итоги исследования.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Загрузка

In [1]:
# импорт бибилиотек и модулей
import pandas as pd
import numpy as np
import catboost as ctb
import lightgbm as lgb
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


In [2]:
# выгрузим файл, выведем основную информацию на экран
df = pd.read_csv('toxic_comments.csv') #

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [3]:
df.sample(10, random_state=1)

,text,toxic
24915,"YOU ARE A FAT, GEEKY PRICK WHO HAS NOTHING TO ...",1
75819,Agent X2: Basically thanks - with a 'little' m...,0
53891,Why are my posts being deleted? \n\nI have tri...,0
154159,"""\n\n Controlled Demolitions and Common Sense ...",0
13040,I do not understand your reply. //Blaxthos ( ...,0
123190,Is this the bizarro world? Removing content is...,0
33626,"Well, WP:RS says that articles should use reli...",0
1150,Oh hear me go someone removes all my pages i g...,0
48633,can't believe this article was deleted\nI'm su...,0
42817,"""\n\n Comments on GamerGate Workshop page \n\n...",0


### Предобработка

In [4]:
# разделим фичи и целевые признаки
features = df['text'].values
target = df['toxic']

#### Очистка, приведение к нижнему регистру

In [5]:
def clear_text(text):   
    """Очищает текст от ненужных символов"""
    a = re.sub(r'[^a-zA-Z ]', ' ', text)
    a = ' '.join(a.split())
    return a

# циклом пройдем по каждой строке вернем очищенный текст
features_cleared = []
n = int(features.shape[0])

for i in range(n):
    features_cleared.append( str.lower(clear_text(features[i])) )

# переведем список в Series
features_cleared = pd.Series( features_cleared ) 

print('\n--------------------\n До обработки \n-------------------- \n',features[1])
print('\n--------------------\n После обработки \n-------------------- \n',features_cleared[1])


--------------------
 До обработки 
-------------------- 
 D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)

--------------------
 После обработки 
-------------------- 
 d aww he matches this background colour i m seemingly stuck with thanks talk january utc


#### Лемматизация текста

In [6]:
%%time

nltk.download('wordnet') # загрузим данные из библиотеки nltk.download('wordnet') для лемматизации

# присвоим объекту wnl структуру данных WordNetLemmatizer()
wnl = WordNetLemmatizer()

def lemma_func(text):
    
    """Функция токенизирует и лемматизирует текст датафрейма, 
        возвращает pd.Series"""
    
    text_lemmatized = []
    
    # счетчик количества предложений
    n = int(text.shape[0])
    
    # циклом пройдем по каждой строке
    for i in range(n):
        # разделим слова по пробелу
        txt_splitted = text[i].split(' ')
        
        # пройдем по каждому слову списка 'txt_splitted', вернем его лемму, соединим в список lemmatized_output
        lemmatized_output = ' '.join([wnl.lemmatize(w) for w in txt_splitted])
    
        # добавим в итоговый список полученной лемматизированное предложение
        text_lemmatized.append(lemmatized_output)
    
    return pd.Series(text_lemmatized)

features_lemmatized = lemma_func(features_cleared)

print("Исходный текст:", features_cleared[1])
print('----------------------------------------------------')
print("Лемматизированный текст:", features_lemmatized[1])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Danil\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Исходный текст: d aww he matches this background colour i m seemingly stuck with thanks talk january utc
----------------------------------------------------
Лемматизированный текст: d aww he match this background colour i m seemingly stuck with thanks talk january utc
Wall time: 1min 5s


#### Разбивка на обучающую, тестовую выборки, скоринг TF-IDF

* разбивка на обучающую, тестовую выборки

In [7]:
X_train, X_test, y_train, y_test = train_test_split(features_lemmatized, target, test_size=0.25, random_state=1)

* скоринг TF-IDF с учетом стоп-слов

In [8]:
# 1) Создадим счётчик
# 2) Чтобы он считал словосочетания, укажем размер N-граммы через аргумент 'ngram_range'
# 3) Чтобы почистить мешок слов, найдём стоп-слова, то есть слова без смысловой нагрузки.
#    3.1) загрузим список стоп-слов:
nltk.download('stopwords')

#    3.2) Вызовем функцию stopwords.words(), передадим ей аргумент 'english', то есть англоязычные стоп-слова:
stop_words = set(stopwords.words('english')) 
#    3.3) передадим список стоп-слов в счётчик TfidfVectorizer():
tf_idf_vect = TfidfVectorizer(stop_words=stop_words)

# Сначала применим TfidfVectorizer() на обучающей выборке и преобразуем в матрицу функций TF-IDF 
# Чтобы посчитать TF-IDF для корпуса текстов, вызовем функцию fit_transform на обучающей выборке:
tf_idf_train = tf_idf_vect.fit_transform(X_train)

# TF-IDF для корпуса текстов test-выбоки, вызовем функцию transform на тестовой выборке:
tf_idf_test = tf_idf_vect.transform(X_test)

print('Размер TF-IDF-массива train:', tf_idf_train.shape)
print('Размер TF-IDF-массива test:', tf_idf_test.shape)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Danil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Размер TF-IDF-массива train: (119678, 133434)
Размер TF-IDF-массива test: (39893, 133434)


**Вывод:**
* Произведена очистка от ненужных символов, приведение к нижнему регистру
* Выполнена лемматизация текста
* Данные разбиты на обучающую, тестовую выборки, произведен скоринг векторов слов методом TF-IDF

## Обучение и предсказания

* Создадим словарь, в который запишем результаты вычислений метрики качества предсказаний на обучающих и тестовых данных

In [9]:
results_dict = {
    'train_set':{'LogisticRegression':list(),
                 'RandomForestClassifier':list(),
                 'CatBoostClassifier':list(),
                 'LGBMClassifier':list()
                },
    'test_set':{'LogisticRegression':list(),
                 'RandomForestClassifier':list(),
                 'CatBoostClassifier':list(),
                 'LGBMClassifier':list()
               }
}

### LogisticRegression

In [10]:
%%time

model = LogisticRegression(solver='liblinear')

model.fit(tf_idf_train, y_train)
pred_train = model.predict(tf_idf_train)

score_train = f1_score(pred_train, y_train)

print('Predictions shape on train set - ', pred_train.shape)
print('F1-score on train set - ', score_train)

Predictions shape on train set -  (119678,)
F1-score on train set -  0.7602147330748583
Wall time: 1.73 s


In [11]:
pred_test = model.predict(tf_idf_test)

score_test = f1_score(pred_test, y_test)

print('Predictions shape on test set - ', pred_test.shape)
print('F1-score on test set - ', score_test)

Predictions shape on test set -  (39893,)
F1-score on test set -  0.7324859592078038


In [12]:
results_dict['train_set']['LogisticRegression'] = np.round(score_train, 3)
results_dict['test_set']['LogisticRegression'] = np.round(score_test, 3)

### RandomForestClassifier

In [13]:
%%time

rnd_frst = RandomForestClassifier(random_state=1)

rnd_frst.fit(tf_idf_train, y_train)
pred_train = rnd_frst.predict(tf_idf_train)

score_train = f1_score(pred_train, y_train)

print('Predictions shape on train set - ', pred_train.shape)
print('F1-score on train set - ', score_train)

Predictions shape on train set -  (119678,)
F1-score on train set -  0.998346970824035
Wall time: 7min 27s


In [14]:
pred_test = rnd_frst.predict(tf_idf_test)

score_test = f1_score(pred_test, y_test)

print('Predictions shape on test set - ', pred_test.shape)
print('F1-score on test set - ', score_test)

Predictions shape on test set -  (39893,)
F1-score on test set -  0.6987364895722332


In [15]:
results_dict['train_set']['RandomForestClassifier'] = np.round(score_train, 3)
results_dict['test_set']['RandomForestClassifier'] = np.round(score_test, 3)

### CatBoostClassifier

In [16]:
%%time

ctb_clf = ctb.CatBoostClassifier(iterations=200, random_state=1)

ctb_clf.fit(tf_idf_train, y_train)
pred_train = ctb_clf.predict(tf_idf_train)

score_train = f1_score(pred_train, y_train)

print('Predictions shape on train set - ', pred_train.shape)
print('F1-score on train set - ', score_train)

Learning rate set to 0.347698
0:	learn: 0.4088747	total: 1.33s	remaining: 4m 25s
1:	learn: 0.3050586	total: 2.36s	remaining: 3m 53s
2:	learn: 0.2618436	total: 3.36s	remaining: 3m 40s
3:	learn: 0.2376257	total: 4.31s	remaining: 3m 31s
4:	learn: 0.2264343	total: 5.26s	remaining: 3m 25s
5:	learn: 0.2187414	total: 6.2s	remaining: 3m 20s
6:	learn: 0.2133123	total: 7.11s	remaining: 3m 15s
7:	learn: 0.2085965	total: 8.07s	remaining: 3m 13s
8:	learn: 0.2037518	total: 8.99s	remaining: 3m 10s
9:	learn: 0.2001982	total: 9.92s	remaining: 3m 8s
10:	learn: 0.1960476	total: 10.9s	remaining: 3m 8s
11:	learn: 0.1933418	total: 11.9s	remaining: 3m 6s
12:	learn: 0.1906938	total: 12.9s	remaining: 3m 5s
13:	learn: 0.1884155	total: 13.9s	remaining: 3m 4s
14:	learn: 0.1863506	total: 14.8s	remaining: 3m 3s
15:	learn: 0.1839878	total: 15.8s	remaining: 3m 1s
16:	learn: 0.1818206	total: 16.7s	remaining: 3m
17:	learn: 0.1800774	total: 17.6s	remaining: 2m 58s
18:	learn: 0.1786581	total: 18.5s	remaining: 2m 56s
19:	

In [17]:
pred_test = ctb_clf.predict(tf_idf_test)

score_test = f1_score(pred_test, y_test)

print('Predictions shape on test set - ', pred_test.shape)
print('F1-score on test set - ', score_test)

Predictions shape on test set -  (39893,)
F1-score on test set -  0.7476020042949177


In [18]:
results_dict['train_set']['CatBoostClassifier'] = np.round(score_train, 3)
results_dict['test_set']['CatBoostClassifier'] = np.round(score_test, 3)

### LGBMClassifier

In [19]:
%%time

lgb_clf = lgb.LGBMClassifier(random_state=1)

lgb_clf.fit(tf_idf_train, y_train)
pred_train = lgb_clf.predict(tf_idf_train)

score_train = f1_score(pred_train, y_train)

print('Predictions shape on train set - ', pred_train.shape)
print('F1-score on train set - ', score_train)

Predictions shape on train set -  (119678,)
F1-score on train set -  0.7780888030888032
Wall time: 29.3 s


In [20]:
pred_test = lgb_clf.predict(tf_idf_test)

score_test = f1_score(pred_test, y_test)

print('Predictions shape on test set - ', pred_test.shape)
print('F1-score on test set - ', score_test)

Predictions shape on test set -  (39893,)
F1-score on test set -  0.7506778935350363


In [21]:
results_dict['train_set']['LGBMClassifier'] = np.round(score_train, 3)
results_dict['test_set']['LGBMClassifier'] = np.round(score_test, 3)

## Итоги исследования

In [22]:
print('Итоговые результаты обучения и тестирования моделей: ')
pd.DataFrame(results_dict)

Итоговые результаты обучения и тестирования моделей: 


,train_set,test_set
LogisticRegression,0.760,0.732
RandomForestClassifier,0.998,0.699
CatBoostClassifier,0.788,0.748
LGBMClassifier,0.778,0.751


**Проект для «Викишоп»** - исследовательский проект, целью которого - построить модель классификации комментариев на позитивные и негативныесо значением метрики качества F1 не меньше 0.75.

Для достижения указанной цели были решены следующие задачи:
1. Загрузите и подготовьте данные.
В рамках подготовки выполнено:
* Очистка от ненужных символов, приведение к нижнему регистру
* Лемматизация текста
* Содан мешок слов с исключением стоп-слов
* Данные разбиты на обучающую, тестовую выборки, произведен скоринг векторов слов методом TF-IDF
2. Обучены разные модели: LogisticRegression, RandomForestClassifier, CatBoostClassifier, LGBMClassifier - и рассчитана итоговая метрика качества F1 на обучающей и тестовой выборки. 
3. Результаты обучения и предсказаний показали, что: 
* все модели оказались переобучены в той или иной степени: необходимо ограничивать степень обучения и подбирать гиперпараметры;
* наилучшие результаты на тестовой выборке показала `LGBMClassifier	(F1-score - 0.751)`
* Стоит отметить, что модели CatBoostClassifier (F1-score - 0.748) и  LogisticRegression тоже показала хорошие результаты (F1-score - 0.732)
* Наихудшей как с точки зрения времени обучения, так и с точки зрения итоговой метрики качества оказалась модель RandomForestClassifier	(F1-score - 0.699).